In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
from sqlalchemy import create_engine, Column, Table, String, MetaData

# --- Define the directory path ---
DB_DIR = "databases" 
DB_FILE = f'sqlite:///{DB_DIR}/cats.db'

# --- 1. Ensure the directory exists ---
if not os.path.exists(DB_DIR):
    # This creates the 'databases' folder if it doesn't exist
    os.makedirs(DB_DIR) 

# --- 2. Create the engine now that the path is guaranteed to exist ---
metadata_obj = MetaData()
engine = create_engine(DB_FILE, future=True)

cat_breeds_table = Table(
    "cat_breeds",
    metadata_obj,
    Column("name", String(16), primary_key=True),
    Column("origin", String(16), nullable=False),
    Column("temperament", String(16), nullable=False),
    Column("size", String(16), nullable=False),
    Column("social_behavior", String(16), nullable=False),
)

metadata_obj.create_all(engine)


In [13]:
from sqlalchemy import insert

breeds_data = [
    {
        'name': 'Savannah',
        'origin': 'USA',
        'temperament': 'high energy',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Ragdoll',
        'origin': 'USA',
        'temperament': 'gentle',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Maine Coon',
        'origin': 'USA (Maine)',
        'temperament': 'friendly',
        'size': 'Very Large',
        'social_behavior': 'sociable'
    }
]
for breed in breeds_data:
    stmt = insert(cat_breeds_table).values(**breed)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [14]:
with engine.connect() as connection:
    result = connection.exec_driver_sql("SELECT * FROM cat_breeds")
    print(result.fetchall())

[('Savannah', 'USA', 'high energy', 'Large', 'sociable'), ('Ragdoll', 'USA', 'gentle', 'Large', 'sociable'), ('Maine Coon', 'USA (Maine)', 'friendly', 'Very Large', 'sociable')]


In [16]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["cat_breeds"])

In [20]:
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

llm = OpenAI(model="gpt-4", temperature=0, verbose=True)
Settings.llm = llm


In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database,
    context_query_kwargs={"cat_breeds": (
        "The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. \n"
        "Do not attempt to run a query if the column is not among available columns.\n"
        "Do not use unexistant colums in a WHERE part of the query. \n"
    )}
)

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'llama_index.core.query_engine.sql'

In [ ]:
openai.log = "debug"

In [ ]:
response = query_engine.query("What kind of a temperament does a Ragdoll cat have?")

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use unexistant colums in a WHERE part of the query. 

> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use unexistant colums in a WHERE part of the query. 



message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed.\\nUse the following format:\\nQuestion: Question here\\nSQLQuery: SQL Query to run\\nSQLResult: Result of the SQLQuery\\nAnswer: Final answer here\\nOnly use the tables listed below.\\nTable \'cat_breeds\' has columns: name (VAR

In [ ]:
print(response)

The Ragdoll cat breed is known for having a gentle temperament.


In [ ]:
response.metadata

{'result': [('gentle',)],
 'sql_query': "SELECT temperament\nFROM cat_breeds\nWHERE name = 'Ragdoll'"}